In [ ]:
from pathlib import Path
import torch
 

import sys 
sys.path.append("/home/franka_deoxys/diffusion_policy")



import os
import pathlib
import click
import hydra
import torch
import dill
import wandb
import json
from diffusion_policy.workspace.base_workspace import BaseWorkspace
from diffusion_policy.env_runner.robomimic_image_runner import RobomimicImageRunner

 
import numpy as np 
import collections 
import tqdm
import h5py
import math 
import wandb.sdk.data_types.video as wv
from diffusion_policy.gym_util.async_vector_env import AsyncVectorEnv
from diffusion_policy.gym_util.sync_vector_env import SyncVectorEnv
from diffusion_policy.gym_util.multistep_wrapper import MultiStepWrapper
from diffusion_policy.gym_util.video_recording_wrapper import VideoRecordingWrapper, VideoRecorder
from diffusion_policy.model.common.rotation_transformer import RotationTransformer

from diffusion_policy.policy.base_image_policy import BaseImagePolicy
from diffusion_policy.common.pytorch_util import dict_apply
from diffusion_policy.env_runner.base_image_runner import BaseImageRunner
from diffusion_policy.env.robomimic.robomimic_image_wrapper import RobomimicImageWrapper
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils 
from diffusion_policy.workspace.train_diffusion_unet_hybrid_workspace import TrainDiffusionUnetHybridWorkspace

import h5py
import matplotlib.pyplot as plt



import numpy as np
import matplotlib.pyplot as plt
import tqdm


import os
from pathlib import Path

import cv2
import numpy as np

sys.path.append("/home/franka_deoxys/deoxys_control/deoxys")

from deoxys import config_root
from deoxys.franka_interface import FrankaInterface
# from deoxys.utils import RobotStateRawObsDictGenerator, YamlConfig
from deoxys.utils import  YamlConfig
from deoxys.utils.config_utils import robot_config_parse_args
from deoxys.utils.input_utils import input2action
from deoxys.utils.io_devices import SpaceMouse
from deoxys.utils.log_utils import get_deoxys_example_logger


from threading import Thread
import time

from collections import OrderedDict

import torch

from deoxys_vision.networking.camera_redis_interface import CameraRedisSubInterface
from deoxys_vision.utils.camera_utils import assert_camera_ref_convention, get_camera_info

np.set_printoptions(precision=3, suppress=True)

logger = get_deoxys_example_logger()

In [ ]:
safe_set_file="Safe_diffusion_policy/safe_set_6d_dummy.npz"
safe_set, hull_equations, _ = load_safe_set(safe_set_file)

### Now run inference in loop

JOINT_POSITION


In [ ]:
# # reset robot
set_gripper(open=True)
reset_joints_to(robot_interface, reset_joint_positions)
set_gripper(open=True)
policy.reset()

images = []
all_poses_7d = []
all_inside_flags = []

for _ in range(50):
    # 1) Acquire user input
    action, grasp = input2action(device=spacemouse, controller_type=controller_type)
    if action is None:
        break
    if len(robot_interface._state_buffer) == 0:
        continue

    # 2) Get observation
    obs_dict = get_current_obs(robot_interface)
    img = obs_dict['eye_in_hand_rgb']
    images.append(img)
    obs = framestacker.add_new_obs(obs_dict)

    # 3) Policy predicts an action (whatever format: delta pose, EEF pose, etc.)
    #    We'll send it to the robot
    action_pred = predict_action(obs)  # shape is up to your policy
    # print("Raw policy prediction:", action_pred)

    

    # 4) Let's say we control the robot with the first 4 predicted actions
    for i, single_cmd in enumerate(action_pred[:4]):
        print(f"Sending to robot: action {i} =>", single_cmd)
        robot_interface.control(
            controller_type=controller_type,
            action=single_cmd,
            controller_cfg=controller_cfg,
        )

        # (Optional) short delay for the robot to move
        time.sleep(0.05)

        # 5) Now read the *actual* real joint angles from the robot
        #    e.g. shape (7,) => [q1, q2, q3, q4, q5, q6, q7]
        js_real = robot_interface.last_q  
        if js_real is None:
            print("No joint angles read. Skipping.")
            continue

        # 6) Convert those real joints => 7D EEF pose => 6D => safe-set check
        eef_pose_7d = joint_angles_to_end_effector_pose(js_real)  # shape (7,)
        eef_pose_6d = pose7d_to_6d(eef_pose_7d)
        inside, distances = is_pose_in_safe_set_6d(eef_pose_6d, hull_equations, tol=1e-1)

        # print(f"   => Real Joints {js_real}, EEF pose {eef_pose_7d}")
        print(f"   => inside={inside}, distances={distances}")

        # 7) Collect for final visualization
        all_poses_7d.append(eef_pose_7d)
        all_inside_flags.append(inside)

robot_interface.close()

JOINT_POSITION
from controller:  [0. 0. 0.] [0. 0. 0.]
Sending to robot: action 0 => [ 0.389  0.     0.096  0.011 -0.009  0.004 -0.996]
   => inside=True, distances=[-0.303 -0.076 -1.054 ... -0.129 -0.13  -0.126]
Sending to robot: action 1 => [ 0.393 -0.001  0.085  0.015 -0.003  0.001 -1.   ]
   => inside=True, distances=[-0.306 -0.075 -1.069 ... -0.141 -0.141 -0.138]
Sending to robot: action 2 => [ 0.427  0.002  0.078  0.008  0.003 -0.003 -0.999]
   => inside=True, distances=[-0.309 -0.072 -1.083 ... -0.154 -0.154 -0.151]
Sending to robot: action 3 => [ 0.435  0.001  0.073  0.006  0.006 -0.004 -1.   ]
   => inside=True, distances=[-0.315 -0.067 -1.104 ... -0.173 -0.173 -0.17 ]
from controller:  [0. 0. 0.] [0. 0. 0.]
Sending to robot: action 0 => [ 0.248 -0.003  0.113  0.008  0.034  0.006 -0.999]
   => inside=True, distances=[-0.326 -0.057 -1.152 ... -0.219 -0.218 -0.216]
Sending to robot: action 1 => [ 0.28  -0.     0.126  0.006  0.03   0.001 -1.   ]
   => inside=True, distances=[-0.3

KeyboardInterrupt: 

In [ ]:
from collections import defaultdict, deque

class FrameStackForTrans:
    def __init__(self, num_frames):
        self.num_frames = num_frames
        self.obs_history = {}
        
    def reset(self, init_obs):
        """ 
        init_obs: dict of initial observation at the start of the episode
        return stacked obs by repeating the first observation num_frames times
        """
        
        self.obs_history = {}
        for k in init_obs:
            self.obs_history[k] = deque([init_obs[k][None] for _ in range(self.num_frames)], maxlen=self.num_frames,)
        obs = { k : np.concatenate(self.obs_history[k], axis=0) for k in self.obs_history }
        return obs 

    def add_new_obs(self, new_obs):
        """ 
        new_obs: dict of new observation at current timestep
        return stacked obs
        """
        for k in new_obs:
            if 'timesteps' in k or 'actions' in k: continue
            self.obs_history[k].append(new_obs[k][None])

        obs= { k : np.concatenate(self.obs_history[k], axis=0) for k in self.obs_history }
        return obs 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class BaseRawObsDictGenerator:
    def __init__(self, *args, **kwargs):
        self.last_obs_dict = None

    def get_raw_obs_dict(self, state_info):
        """
        Args:
           state_info (dict): A dictionary of robot state + images
        """
        obs_dict = {}
        raise NotImplementedError

    def load(self):
        raise NotImplementedError


class RobotStateRawObsDictGenerator(BaseRawObsDictGenerator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def load(self, obs_dict, key, value, check_valid=True):
        """
        This is to check if the data is correct or not. Sometimes the data will be all zero depending on the networking conditions.
        """
        if (
            (
                np.sum(np.abs(value)) == 0.0
                and key in ["ee_states", "joint_states", "gripper_states"]
            )
            and check_valid
            and self.last_obs_dict is not None
        ):
            value = self.last_obs_dict[key]
        obs_dict[key] = value

    def get_raw_obs_dict(self, state_info):
        last_state = state_info["last_state"]
        last_gripper_state = state_info["last_gripper_state"]
        obs_dict = {}

        ee_states = np.array(last_state.O_T_EE)
        joint_states = np.array(last_state.q)
        gripper_states = np.array([last_gripper_state.width])

        self.load(obs_dict, "ee_states", ee_states)
        self.load(obs_dict, "joint_states", joint_states)
        # Gripper widh will probably become zero
        self.load(obs_dict, "gripper_states", gripper_states, check_valid=False)

        for state in ["ee_states", "joint_states", "gripper_states"]:
            if (
                np.sum(np.abs(obs_dict[state])) <= 1e-6
                and self.last_obs_dict is not None
            ):
                print(f"{state} missing!!!!")
                obs_dict[state] = self.last_obs_dict[state]
        self.last_obs_dict = obs_dict
        return obs_dict

In [ ]:
logger.warn("This is a very dummy control policy!!!!!")

args = robot_config_parse_args()
robot_interface = FrankaInterface(os.path.join(config_root, args.interface_cfg))
controller_cfg = YamlConfig(
    os.path.join(config_root, args.controller_cfg)
).as_easydict()
controller_type = args.controller_type

spacemouse = SpaceMouse(vendor_id=9583, product_id=50734)
spacemouse.start_control()

raw_obs_dict_generator = RobotStateRawObsDictGenerator()

import time

time.sleep(0.3)
dummy_torch_model = torch.nn.Linear(7, 1)

In [ ]:
def set_gripper(open=True):
    d=-1. if open else 1.0
    action_close=np.array([ 0.,  0., -0.,  0.,  0., -0., d])
    robot_interface.control(
        controller_type=controller_type,
        action=action_close,
        controller_cfg=controller_cfg,
    )

In [ ]:
# logger.info("Resetting to the initial configuration")
# reset_joints_to(robot_interface, joint_sequence[0])
set_gripper(open=True)

In [ ]:
camera_ids = [0, 1]
# camera_ids = [0]
cr_interfaces = {}

use_depth = False
for camera_id in camera_ids:
    camera_ref=f"rs_{camera_id}"
    assert_camera_ref_convention(camera_ref)
    camera_info = get_camera_info(camera_ref)
    print('---------****-----------')
    print(camera_info)
    print('--------------------------')
    # camera_info=  {'camera_id': 0, 'camera_type': 'rs', 'camera_name': 'camera_rs_0'}
# 

    cr_interface = CameraRedisSubInterface(camera_info=camera_info, use_depth=use_depth, redis_host='127.0.0.1')
    cr_interface.start()
    cr_interfaces[camera_id] = cr_interface

In [ ]:
def get_imgs(use_depth=False):
    data = {}
    for camera_id in camera_ids:
        img_info = cr_interfaces[camera_id].get_img_info()
        data[f"camera_{camera_id}"]=img_info

        imgs = cr_interfaces[camera_id].get_img()
    
        color_img = imgs["color"][..., ::-1]

        # color_img = cv2.resize(color_img, (128, 128))  
        # depth_img = cv2.resize(depth_img, (128, 128))

        # color_img = cv2.resize(color_img, (224, 224))  
        color_img = cv2.resize(color_img, None, fx=0.5, fy=0.5)

        data[f"camera_{camera_id}_color"]=color_img

        if use_depth:
            depth_img = imgs["depth"]
            depth_img = cv2.resize(depth_img, (224, 224)) 
            data[f"camera_{camera_id}_depth"]=depth_img
    return data

In [ ]:
data = get_imgs()
data.keys()

In [ ]:
data = get_imgs()
img_wrist = data['camera_0_color']
img_front = data['camera_1_color']

agentview_rgb=img_wrist 
eye_in_hand_rgb=img_front
agentview_rgb.shape, eye_in_hand_rgb.shape

In [ ]:
plt.subplot(121)
plt.imshow(agentview_rgb[:,:,::-1])
plt.subplot(122)
plt.imshow(eye_in_hand_rgb[:,:,::-1])

In [ ]:
def get_current_obs(robot_interface):
    last_state = robot_interface._state_buffer[-1]
    last_gripper_state = robot_interface._gripper_state_buffer[-1]

    obs_dict = raw_obs_dict_generator.get_raw_obs_dict(
    {"last_state": last_state, "last_gripper_state": last_gripper_state})
    # obs_dict={} #rgb only 

    data = get_imgs()
    img_wrist = data['camera_0_color']
    img_front = data['camera_1_color']

    agentview_rgb=img_wrist
    eye_in_hand_rgb=img_front
    # agentview_rgb=cv2.resize(img_wrist, (120, 120))  
    # eye_in_hand_rgb=cv2.resize(img_front, (120, 120)) 

    # agentview_rgb.shape, eye_in_hand_rgb.shape

    obs_dict['agentview_rgb']=agentview_rgb.transpose(2, 1, 0)
    obs_dict['eye_in_hand_rgb']=eye_in_hand_rgb.transpose(2, 1, 0)
    return obs_dict

In [ ]:
obs_dict=get_current_obs(robot_interface)
for key in obs_dict.keys():
    obs_dict[key]=obs_dict[key][None]
    print(key, obs_dict[key].shape)

### Now load dp policy

In [ ]:
checkpoint = "/home/franka_deoxys/data_franka/dp_policy/coffee_after_train.ckpt"

payload = torch.load(open(checkpoint, 'rb'), pickle_module=dill)
cfg = payload['cfg'] 
workspace = TrainDiffusionUnetHybridWorkspace(cfg, output_dir=None)
workspace: BaseWorkspace
workspace.load_payload(payload, exclude_keys=None, include_keys=None)

In [ ]:
# get policy from workspace
policy = workspace.model
if cfg.training.use_ema:
    policy = workspace.ema_model

device = torch.device(device)
policy.to(device)
policy.eval()
pass 

In [ ]:
rotation_transformer = RotationTransformer('axis_angle', 'rotation_6d')

def undo_transform_action( action):
    raw_shape = action.shape
    if raw_shape[-1] == 20:
        # dual arm
        action = action.reshape(-1,2,10)

    d_rot = action.shape[-1] - 4
    pos = action[...,:3]
    rot = action[...,3:3+d_rot]
    gripper = action[...,[-1]]
    rot = rotation_transformer.inverse(rot)
    uaction = np.concatenate([
        pos, rot, gripper
    ], axis=-1)

    if raw_shape[-1] == 20:
        # dual arm
        uaction = uaction.reshape(*raw_shape[:-1], 14)

    return uaction


In [ ]:
# dataset_path="/media/franka_deoxys/DATA/demo_rss_paper/dec20/hdfs/ola_114.hdf5"

# f= h5py.File(dataset_path, 'r')
# demos= f['data']
# len(demos)

In [ ]:
# demo=f['data']['demo_1']
# actions_org=demo['actions']
# obss=demo['obs']
# actions_org.shape

In [ ]:
def predict_action(obs):
    """ 
    obs: 2x...
    """
    np_obs_dict = {key:obs[key] for key in keys_select}
    obs_dict = dict_apply(np_obs_dict, 
        lambda x: torch.from_numpy(x).to(
            device=device))

    for key in obs_dict.keys():
        obs_dict[key]=obs_dict[key].unsqueeze(0) 

    
    with torch.no_grad():
        action_dict = policy.predict_action(obs_dict)

    # device_transfer
    np_action_dict = dict_apply(action_dict, lambda x: x.detach().to('cpu').numpy())


    # step env
    env_action =  np_action_dict['action']
    env_action = undo_transform_action(env_action)

    env_action=env_action.squeeze()
    return env_action

In [ ]:
framestacker=FrameStackForTrans(2)
keys_select = ['agentview_rgb', 'joint_states', 'ee_states', 'eye_in_hand_rgb', 'gripper_states']

In [ ]:
# i=0
# obs = {k: obss[k][i] for k in obss.keys()}
# obs['agentview_rgb'] = obs['agentview_rgb'].transpose(2,0,1)
# obs['eye_in_hand_rgb'] = obs['eye_in_hand_rgb'].transpose(2,0, 1)

# for k in obs.keys():
#     print(k, obs[k].shape)

In [ ]:
obs_dict=get_current_obs(robot_interface)
for key in obs_dict.keys(): 
    print(key, obs_dict[key].shape)

In [ ]:
# action_pred=predict_action(obs)

In [ ]:
# action_pred.shape

In [ ]:
framestacker=FrameStackForTrans(2) 
policy.reset()

obs_dict=get_current_obs(robot_interface)
obs = framestacker.reset(obs_dict)

In [ ]:
obs_dict=get_current_obs(robot_interface)
obs = framestacker.add_new_obs(obs_dict)
action_pred=predict_action(obs)

In [ ]:
action_pred

In [ ]:
for act in action_pred:  
    print(act.shape)

In [ ]:
policy.reset()
 
for _ in range(500):
    action, grasp = input2action(
        device=spacemouse,
        controller_type=controller_type,
    )
    if action is None:
        break
    if len(robot_interface._state_buffer) == 0:
        continue

    obs_dict=get_current_obs(robot_interface)
    obs = framestacker.add_new_obs(obs_dict)
    action_pred=predict_action(obs)

    # print(f'actions: {action[0]:.3f} {action[1]:.3f} {action[2]:.3f} {action[-1]:.3f}')
    
    for action in action_pred:  
        robot_interface.control(
            controller_type=controller_type,
            action=action,
            controller_cfg=controller_cfg,
        )

    # time.sleep(0.045)

    
robot_interface.close()